In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os
from pathlib import Path
import csv
import pandas as pd

In [2]:
NORMAL_FOLDER = "csv_cwru_data/normal"
DEFECT_FOLDER = "csv_cwru_data/defect"
SAVE_NORMAL_FOLDER = "data_cwru_train/normal"
SAVE_DEFECT_FOLDER = "data_cwru_train/defect"
SAMPLING_RATE = 12000
FRAME = 512
NOVERLAP = FRAME // 2 

In [3]:
def create_and_save_spectrogram(signal_data, sampling_rate, save_path, nperseg, noverlap):
    """
    Создает спектрограмму из сигнала и сохраняет ее как изображение.
    
    Аргументы:
    signal_data (np.array): Входной одномерный сигнал (временной ряд).
    sampling_rate (int): Частота дискретизации сигнала в Гц.
    save_path (str): Полный путь для сохранения файла (включая имя файла, например, 'data/normal/spec_01.png').
    """

    # Создаем спектрограмму
    frequencies, times, Zxx = signal.stft(signal_data, fs=sampling_rate, nperseg=nperseg, noverlap=noverlap)
    dpi = 256
    img_size_px = 1920 
    figsize_inch = img_size_px / dpi
    
    plt.figure(figsize=(figsize_inch, figsize_inch))
    
    # Отображаем спектрограмму
    plt.pcolormesh(times, frequencies, np.log(np.abs(Zxx) + 1e-9), cmap='gray')
    
    # Убираем все лишнее
    plt.axis('off')
    plt.gca().set_position([0, 0, 1, 1])
    
    # Создаем директорию
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    # Сохраняем файл с заданным DPI
    plt.savefig(save_path, dpi=dpi)
    
    plt.close()

In [4]:
def read_data(file_path):
    with open(file_path, mode='r', newline='') as csv_file:
        df = pd.read_csv(file_path)
        signal = df['vibration'].values
        return signal

In [5]:
for file in Path(NORMAL_FOLDER).iterdir():
    if file.is_file():
        data = read_data(file)
        name = file.name.split(".")[:-1][0]
        save_path = SAVE_NORMAL_FOLDER + "/" + name + ".png"
        create_and_save_spectrogram(data, SAMPLING_RATE, save_path, FRAME, NOVERLAP)

In [6]:
for file in Path(DEFECT_FOLDER).iterdir():
    if file.is_file():
        data = read_data(file)
        name = file.name.split(".")[:-1][0]
        save_path = SAVE_DEFECT_FOLDER + "/" + name + ".png"
        create_and_save_spectrogram(data, SAMPLING_RATE, save_path, FRAME, NOVERLAP)